In [1]:
from pyspark.sql import SparkSession

DIRPATH = "C:/Users/sanjay s risbud/Downloads/sundog/"
spark = SparkSession.builder.master("local").appName("Exercise").getOrCreate()

In [2]:
from pyspark.sql.functions import format_number
df = spark.read.csv(DIRPATH + "1800.csv", inferSchema=True)
df = df.filter((df["_c2"] == "TMIN") | (df["_c2"] == "TMAX"))\
        .withColumn("fahrenheit", format_number(((df["_c3"] * 0.1 * 9/5) + 32), 2).cast("float"))
df.show()

+-----------+--------+----+----+----+----+---+----+----------+
|        _c0|     _c1| _c2| _c3| _c4| _c5|_c6| _c7|fahrenheit|
+-----------+--------+----+----+----+----+---+----+----------+
|ITE00100554|18000101|TMAX| -75|null|null|  E|null|      18.5|
|ITE00100554|18000101|TMIN|-148|null|null|  E|null|      5.36|
|EZE00100082|18000101|TMAX| -86|null|null|  E|null|     16.52|
|EZE00100082|18000101|TMIN|-135|null|null|  E|null|       7.7|
|ITE00100554|18000102|TMAX| -60|null|   I|  E|null|      21.2|
|ITE00100554|18000102|TMIN|-125|null|null|  E|null|       9.5|
|EZE00100082|18000102|TMAX| -44|null|null|  E|null|     24.08|
|EZE00100082|18000102|TMIN|-130|null|null|  E|null|       8.6|
|ITE00100554|18000103|TMAX| -23|null|null|  E|null|     27.86|
|ITE00100554|18000103|TMIN| -46|null|   I|  E|null|     23.72|
|EZE00100082|18000103|TMAX| -10|null|null|  E|null|      30.2|
|EZE00100082|18000103|TMIN| -73|null|null|  E|null|     18.86|
|ITE00100554|18000104|TMAX|   0|null|null|  E|null|    

In [3]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- fahrenheit: float (nullable = true)



In [4]:
columns = [f"_c{i}" for i in range(2, 8)]
df = df.drop(*columns).withColumnRenamed("_c0", "station_id").withColumnRenamed("_c1", "date")
df.show()
df.show()

+-----------+--------+----------+
| station_id|    date|fahrenheit|
+-----------+--------+----------+
|ITE00100554|18000101|      18.5|
|ITE00100554|18000101|      5.36|
|EZE00100082|18000101|     16.52|
|EZE00100082|18000101|       7.7|
|ITE00100554|18000102|      21.2|
|ITE00100554|18000102|       9.5|
|EZE00100082|18000102|     24.08|
|EZE00100082|18000102|       8.6|
|ITE00100554|18000103|     27.86|
|ITE00100554|18000103|     23.72|
|EZE00100082|18000103|      30.2|
|EZE00100082|18000103|     18.86|
|ITE00100554|18000104|      32.0|
|ITE00100554|18000104|     29.66|
|EZE00100082|18000104|      22.1|
|EZE00100082|18000104|     18.68|
|ITE00100554|18000105|      33.8|
|ITE00100554|18000105|     30.92|
|EZE00100082|18000105|      24.8|
|EZE00100082|18000105|     21.56|
+-----------+--------+----------+
only showing top 20 rows

+-----------+--------+----------+
| station_id|    date|fahrenheit|
+-----------+--------+----------+
|ITE00100554|18000101|      18.5|
|ITE00100554|18000101|

In [5]:
from pyspark.sql.functions import min, max
grouped_df = df.groupBy("station_id").agg(min(df["fahrenheit"]).alias("low"), max(df["fahrenheit"]).alias("high"))
grouped_df.show()

+-----------+----+-----+
| station_id| low| high|
+-----------+----+-----+
|ITE00100554|5.36|90.14|
|EZE00100082| 7.7|90.14|
+-----------+----+-----+



In [6]:
from pyspark.sql.functions import lit
low_condition = [df["station_id"] == grouped_df["station_id"], df["fahrenheit"] == grouped_df["low"]]
high_condition = [df["station_id"] == grouped_df["station_id"], df["fahrenheit"] == grouped_df["high"]]
low_df = grouped_df.join(df, on=low_condition).drop(df["station_id"]).withColumn("high", lit(None))
high_df = grouped_df.join(df, on=high_condition).drop(df["station_id"]).withColumn("low", lit(None))
low_df.unionAll(high_df).show()

+----+-----+-----------+--------+----------+
| low| high| station_id|    date|fahrenheit|
+----+-----+-----------+--------+----------+
|5.36| null|ITE00100554|18000101|      5.36|
| 7.7| null|EZE00100082|18000101|       7.7|
|null|90.14|EZE00100082|18000818|     90.14|
|null|90.14|ITE00100554|18000819|     90.14|
+----+-----+-----------+--------+----------+

